## Google Places API 


### Setup & Auth

In [1]:
import googlemaps
import time
import jsonlines
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
places_api_key = os.getenv("PLACES_API_KEY")

# Initiate Google Maps API client
gmaps = googlemaps.Client(key=places_api_key)


### Pull data from Places API

In [2]:
# Search for places using query 
response = gmaps.places(query='kopi in orchard')

# Parse results & normalise into df
results_json = response.get('results')
df1 = pd.json_normalize(results_json)
df1.head()

,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,price_level,rating,...,user_ratings_total,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code
0,OPERATIONAL,"238 Orchard Turn B3-59 ION Orchard, Singapore ...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Starbucks,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJyRIV6Y0Z2jER4IB6WdnCOTo,2.0,4.2,...,362,1.304086,103.831557,1.305333,103.832817,1.302633,103.830118,True,8R3J+JJ Singapore,6PH58R3J+JJ
1,OPERATIONAL,"435 Orchard Rd, #02-18, Singapore 238877",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,The Coffee Bean and Tea Leaf™ - Wisma Atria,"[{'height': 1440, 'html_attributions': ['<a hr...",ChIJ-UcIQZ4Z2jER25GdthHiP68,2.0,4.2,...,74,1.303715,103.833264,1.305063,103.834400,1.302364,103.831700,True,8R3M+F8 Singapore,6PH58R3M+F8
2,OPERATIONAL,"2 Orchard Turn #01-15/16 ION Orchard Mall, Sin...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Bacha Coffee ION Orchard,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJw1EsKngZ2jERKHmHIQxZ-jY,NaN,4.2,...,529,1.304130,103.831450,1.305336,103.832752,1.302637,103.830053,True,8R3J+MH Singapore,6PH58R3J+MH
3,OPERATIONAL,"2 Orchard Turn, #B3, #22, Singapore 238801",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Hoshino Coffee @ ION Orchard,"[{'height': 4248, 'html_attributions': ['<a hr...",ChIJ06MdvY0Z2jERx4PGt3RxdP4,NaN,3.7,...,764,1.303929,103.831951,1.304992,103.833323,1.302292,103.830623,True,8R3J+HQ Singapore,6PH58R3J+HQ
4,OPERATIONAL,"277 Orchard Rd, #01-02 Orchard, Singapore 238858",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Flash Coffee,"[{'height': 2500, 'html_attributions': ['<a hr...",ChIJtTPxX0EZ2jERJKaUmQLt4z8,NaN,3.8,...,43,1.300459,103.839352,1.301766,103.840691,1.299066,103.837992,True,8R2Q+5P Singapore,6PH58R2Q+5P


In [6]:
# Each place search query returns a maximum of 60 results per query 
# Each request returns 20 results. If there are more results, a next_page_token is present in the response
# Hence, a function is needed to return all 60 results in a query
def get_all_query_results(query):
    """
    Get maximum number of results (60) for Google Place API Text Search
    """
    json_list = []

    # Search for places using query and save value of next_page_token
    i = 1
    response = gmaps.places(query=query)
    json_list.append(response.get('results'))

    next_page_token = response.get('next_page_token')

    while next_page_token is not None:
        time.sleep(2)
        i += 1 
        print('next_page_token found. Attempting Request #', i, 'for', query)
        response = gmaps.places(page_token=next_page_token)
        print('Request #', i, 'Success!')
        json_list.append(response.get('results'))
        next_page_token = response.get('next_page_token')
    else:
        print('next_page_token not found. Returning jsons for', query)

    return json_list

# Prepare a list of queries for Place Search using a a list of queries & list of locations

def prepare_queries(query_list, location_list):
    """
    Prepare a list of queries for Place Search 

    Parameters:
        query_list (list): A list of queries to search for. Eg ["coffee", "coffee shops", "kopi"]
        location_list (list): A list of locations to search in for each query in query_list. Eg ["bishan", "orchard", "ang mo kio"]

    Returns:
        prepared_query_list (list):  
    """

    prepared_query_list = [i+' in '+j for i in query_list for j in location_list]
    
    return prepared_query_list

In [36]:
# Prepare list of queries using prepare_queries_function
prepared_query_list = prepare_queries(["coffee", "coffee shops", "kopi"],  ["bishan", "orchard", "ang mo kio"])

# Get place result for multiple queries
query_list = prepared_query_list

df_list = []

for query in query_list:
    df = get_all_query_results(query)
    df_list.append(df)

df2 = pd.concat(df_list, axis=0, ignore_index=True)
df2.info()

Attempting Request # 1
next_page_token found. Attempting Request # 2 for coffee in bishan
Request # 2 Success!
next_page_token found. Attempting Request # 3 for coffee in bishan
Request # 3 Success!
next_page_token not found. Returning list of df for coffee in bishan
Attempting Request # 1
next_page_token found. Attempting Request # 2 for coffee in orchard
Request # 2 Success!
next_page_token found. Attempting Request # 3 for coffee in orchard
Request # 3 Success!
next_page_token not found. Returning list of df for coffee in orchard
Attempting Request # 1
next_page_token found. Attempting Request # 2 for coffee in ang mo kio
Request # 2 Success!
next_page_token found. Attempting Request # 3 for coffee in ang mo kio
Request # 3 Success!
next_page_token not found. Returning list of df for coffee in ang mo kio
Attempting Request # 1
next_page_token found. Attempting Request # 2 for coffee shops in bishan
Request # 2 Success!
next_page_token found. Attempting Request # 3 for coffee shops i

### Get place info 

In [10]:
# Get place info using place_id
place_id = df1.loc[1, 'place_id']

# Convert reviews from place into df
response = gmaps.place(place_id)
df3 = pd.json_normalize(response.get('result').get('reviews'))


df3.iloc[:, 4:].head()

,rating,relative_time_description,text,time
0,5,a month ago,"Very cozy place, very attentive staffs.\n\nWe ...",1654352312
1,5,a month ago,Have not been here for a long while. They have...,1653630380
2,4,3 weeks ago,"Simply put, nice chocolate cake, flavourful dr...",1656177484
3,5,2 weeks ago,Maria was excellent in service. She made THE d...,1657035756
4,1,3 months ago,I wonder how these cafes manage to survive? W...,1650467676


In [14]:
# Get list of place IDs from df1
place_id_list = df1.loc[1:3, 'place_id']

df4_list = []

i = 1

# Loop through list of place IDs and request for reviews.
# Normalise reviews for each place IDs as a df
for place_id in place_id_list:
    response = gmaps.place(place_id)
    reviews_df = pd.json_normalize(response.get('result').get('reviews'))
    reviews_df['place_id'] = response.get('result').get('place_id')

    # Append reviews df of each place ID into a list
    df4_list.append(reviews_df)
    i += 1

df4 = pd.concat(df4_list)
df4.iloc[:, 4:].head()

,rating,relative_time_description,text,time,place_id
0,5,a month ago,"Very cozy place, very attentive staffs.\n\nWe ...",1654352312,ChIJ-UcIQZ4Z2jER25GdthHiP68
1,5,a month ago,Have not been here for a long while. They have...,1653630380,ChIJ-UcIQZ4Z2jER25GdthHiP68
2,4,3 weeks ago,"Simply put, nice chocolate cake, flavourful dr...",1656177484,ChIJ-UcIQZ4Z2jER25GdthHiP68
3,5,2 weeks ago,Maria was excellent in service. She made THE d...,1657035756,ChIJ-UcIQZ4Z2jER25GdthHiP68
4,1,3 months ago,I wonder how these cafes manage to survive? W...,1650467676,ChIJ-UcIQZ4Z2jER25GdthHiP68


## Appendix

In [ ]:
types_dummies = df1['types'].str.join('|').str.get_dummies()

df1[['name']].merge(types_dummies, left_index=True, right_index=True)

,name,bakery,bar,cafe,establishment,food,point_of_interest,restaurant,store
0,Killiney Kopitiam - Lucky Plaza,0,0,1,1,1,1,1,1
1,Alchemist Design Orchard,0,0,1,1,1,1,0,1
2,Killiney Kopitiam,0,0,1,1,1,1,0,1
3,Starbucks,0,0,1,1,1,1,0,1
4,Ya Kun Kaya Toast,0,0,1,1,1,1,0,1
5,Kopi & Tarts,0,0,1,1,1,1,0,0
6,The Coffee Bean and Tea Leaf™ - Wisma Atria,0,0,1,1,1,1,0,1
7,Kopi & Roti,0,0,1,1,1,1,0,0
8,% Arabica Singapore 313@Somerset,0,0,1,1,1,1,0,1
9,Toast Box,0,0,1,1,1,1,0,1
